# Fusing LM with Whisper for lower WER
The aim is to fuse a BPE-level LM scores with the generated tokens scores while beam-search decoding in Whisper.

## **MILESTONE 1**:
Instantiate a Language Model to be integrated with Whisper.
The chosen LM is an n-gram language model, trained with [KenLM](https://github.com/kpu/kenlm) library.

### Step 1:
Write code to run an already available LM in a standalone manner and be able to give a score any input sequence.
**Chosen Model**: [Riva ASR Hindi LM](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/speechtotext_hi_in_lm/files?version=deployable_v3.1)


Download and build the KenLM toolkit

In [ ]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
!ls kenlm/build/bin

--2023-11-04 13:14:44--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 480.36K  1.68MB/s    in 0.3s    

2023-11-04 13:14:45 (1.68 MB/s) - written to stdout [491888/491888]

CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detectin

Download the KenLM Python library

❗❗**Don't forget to restart the runtime after running this cell** ❗❗


In [ ]:
!pip install https://github.com/kpu/kenlm/archive/master.zip

     \ 553.6 kB 3.8 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp310-cp310-linux_x86_64.whl size=3184306 sha256=6a7e20994a5b1af9df08981a429943e3ea93f0c66de5ea8750ce83297ba946de
  Stored in directory: /tmp/pip-ephem-wheel-cache-_66qe1hq/wheels/a5/73/ee/670fbd0cee8f6f0b21d10987cb042291e662e26e1a07026462
Successfully built kenlm


In [ ]:
# Download the binary LM from Gdrive
import gdown
url = "https://drive.google.com/uc?id=1-AspJVZRXcrFMuLKx8C4N7uo9PnQytcB"
output = "language_model_3p0.bin"
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1-AspJVZRXcrFMuLKx8C4N7uo9PnQytcB
To: /content/language_model_3p0.bin
100%|██████████| 280M/280M [00:03<00:00, 78.8MB/s]


'language_model_3p0.bin'

In [ ]:
import kenlm
model = kenlm.LanguageModel('/content/language_model_3p0.bin')
print("This is a {}-gram model".format(model.order))

This is a 4-gram model


In [ ]:
# Below are 2 pairs of sentences that sound exactly the same in hindi but one of them is incorrect (lexically or semantically)
# Generated using Bing Chat
book_correct = "मुझे यह किताब पसंद है।"
book_incorrect = "मुझे यह किताब पसन्द है।"

correct_score = model.score(book_correct)
incorrect_score = model.score(book_incorrect)
assert correct_score > incorrect_score
print(correct_score, incorrect_score)

-20.935253143310547 -21.663846969604492


In [ ]:
sings_correct = "वह बहुत अच्छा गाता है।"
sings_incorrect = "वह बहुत अच्छा घाता है।"


correct_score = model.score(sings_correct)
incorrect_score = model.score(sings_incorrect)
assert correct_score > incorrect_score
print(correct_score, incorrect_score)

-19.827430725097656 -22.76061248779297


In [ ]:
# download the original arpa LM file for inspection
url = "https://drive.google.com/uc?id=1-4xQ3YCtsyONtpccGjOD1s9FtHqBX7RL"
output = "language_model_3p0.arpa"
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1-4xQ3YCtsyONtpccGjOD1s9FtHqBX7RL
To: /content/language_model_3p0.arpa
100%|██████████| 3.18G/3.18G [00:36<00:00, 88.3MB/s]


'language_model_3p0.arpa'

In [ ]:
# inspect the last 20 lines inside the LM source
!tail -20 language_model_3p0.arpa

-1.2548329	<s> दोनों ही झल्लाये
-0.2760634	<s> चौधरी के अशुभचिंतकों
-0.04973512	डालियों पर बैठी शुकमंडली
-0.07060567	मनुष्यों को उन्हें बेमुरौवत
-0.049646165	और कड़क कर बोलेमेरी
-0.04038189	निराश हो कर कहानहीं
-0.08863469	पड़ते ही वह अव्यवस्थितचित्त
-0.19321889	दोनों पक्षों से सवालजवाब
-0.051110353	झगड़ू साहु ने कहासमझू
-0.20675866	करें तो उनकी भलमनसी
-0.04876329	नीति को सराहता थाइसे
-0.06436408	<s> मित्रता की मुरझायी
-0.23735626	की गहराई से उपजतें
-0.17502813	पूर्णता की ओर बढातें
-0.18197767	जहाँ से अच्छा हिन्दोसिताँ
-0.04437429	हैं इसकी यह गुलसिताँ
-0.06926097	संतरी हमारा वह पासबाँ
-0.09434804	जिनके दम से रश्कएजनाँ

\end\


In [ ]:
# some useful KenLM commands for future reference
# generate binary
# !kenlm/build/bin/build_binary dataset_tokenized_3gram.arpa dataset_tokenized_3gram.binary
# create a new LM
# !kenlm/build/bin/lmplz -o 3 --text dataset_tokenized.txt --arpa dataset_tokenized_3gram.arpa --discount_fallback
# !tail -20 dataset_tokenized_3gram.arpa

# Integrating the LM with Whisper

In [ ]:
# !pip install https://github.com/HKAB/whisper/archive/master.zip
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.1/153.1 kB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6

In [ ]:
import whisper
import torch
import kenlm

In [ ]:
model = whisper.load_model("small")

In [ ]:
# Download a sample audio file from Gdrive
import gdown
url = "https://drive.google.com/uc?id=1kKeSvrZo8z5Rsp1q-h3GXpG7vHctKMcG"
output = "sample.wav"
gdown.download(url, output, quiet=False)

transcription = "ब्रूड बॉक्स लैंगस्ट्रॉथ छत्ते का एक अनिवार्य हिस्सा है।"

Downloading...
From: https://drive.google.com/uc?id=1kKeSvrZo8z5Rsp1q-h3GXpG7vHctKMcG
To: /content/sample.wav
100%|██████████| 197k/197k [00:00<00:00, 84.9MB/s]


In [ ]:
audio = whisper.load_audio("/content/sample.wav")
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(model.device)


## Baseline Whisper
Without nbest or LM integration

In [ ]:

options = whisper.DecodingOptions(fp16 = False, beam_size=5, without_timestamps=True)
result = whisper.decode(model, mel, options)
baseline = result.text
baseline

'ब्रूद बाँक्ष लंश्टोट छत्ते का एक अनिवार्य हिस्सा है'

# Decoding with LM integration

In [ ]:
# adding the LM
import gdown
url = "https://drive.google.com/uc?id=1-AspJVZRXcrFMuLKx8C4N7uo9PnQytcB"
output = "language_model_3p0.bin"
gdown.download(url, output, quiet=False)

In [ ]:
options = whisper.DecodingOptions(fp16 = False, withlm=True, beam_size=5,
        patience=1.0, lm_path="/content/language_model_3p0.bin", lm_alpha=1.0, lm_beta=0.0,
        without_timestamps=True)
decoding_withLM = whisper.decode(model, mel, options)


In [ ]:
decoding_withLM.text

'ब्रूद बाँक्ष लंश्टोट छत्ते का एक अनिवार्य हिस्सा है'

# Decoding with nbest

In [ ]:
options = whisper.DecodingOptions(fp16 = False, beam_size=5, nbest = True, without_timestamps=True)
nbest = whisper.decode(model, mel, options)

for candidate in nbest:
  print(candidate.text, candidate.avg_logprob)

ब्रूद बाँक्ष लंश्टोट छत्ते का एक अनिवार्य हिस्सा है -0.45290601664576036
ब्रूद बाँक्ष लांश्टोट छत्ते का एक अनिवार्य हिस्सा है -0.4552239240226099
ब्रूद बाँक्ष लंश्टोट छत्ते का एक अनिवार्य हिस्सा है. -0.4729596477443889
ब्रूद बाँक्ष लंश्टोट छत्ते का एक अनिवार्य हिस्सा हैं -0.4831788014557402
ब्रूद बाँक्ष लांश्टोट छत्टे का एक अनिवार्य हिस्सा है -0.46668367385864257


In [ ]:
assert nbest[0].text == baseline

### Adding LM rescoring

In [ ]:
lm_model = kenlm.LanguageModel('/content/language_model_3p0.bin')
print("This is a {}-gram model".format(lm_model.order))

This is a 4-gram model


In [ ]:
nbest_with_lm_score = [(c.text, c.avg_logprob, lm_model.score(c.text)) for c in nbest]
nbest_with_lm_score

[('ब्रूद बाँक्ष लंश्टोट छत्ते का एक अनिवार्य हिस्सा है',
  -0.45290601664576036,
  -36.06303787231445),
 ('ब्रूद बाँक्ष लांश्टोट छत्ते का एक अनिवार्य हिस्सा है',
  -0.4552239240226099,
  -36.06303787231445),
 ('ब्रूद बाँक्ष लंश्टोट छत्ते का एक अनिवार्य हिस्सा है.',
  -0.4729596477443889,
  -44.10427474975586),
 ('ब्रूद बाँक्ष लंश्टोट छत्ते का एक अनिवार्य हिस्सा हैं',
  -0.4831788014557402,
  -37.30411148071289),
 ('ब्रूद बाँक्ष लांश्टोट छत्टे का एक अनिवार्य हिस्सा है',
  -0.46668367385864257,
  -37.54087829589844)]

In [ ]:
lm_weight = 0.01
combined_scores = [(text, whisper_score + lm_score*lm_weight) for text, whisper_score, lm_score in nbest_with_lm_score]
combined_scores.sort(key=lambda t: t[1], reverse=True)
combined_scores

[('ब्रूद बाँक्ष लंश्टोट छत्ते का एक अनिवार्य हिस्सा है', -0.8135363953689049),
 ('ब्रूद बाँक्ष लांश्टोट छत्ते का एक अनिवार्य हिस्सा है', -0.8158543027457544),
 ('ब्रूद बाँक्ष लांश्टोट छत्टे का एक अनिवार्य हिस्सा है', -0.8420924568176269),
 ('ब्रूद बाँक्ष लंश्टोट छत्ते का एक अनिवार्य हिस्सा हैं', -0.8562199162628692),
 ('ब्रूद बाँक्ष लंश्टोट छत्ते का एक अनिवार्य हिस्सा है.', -0.9140023952419476)]

## Importing a HF finetuned model

In [ ]:
!pip install openai-whisper
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 98.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.1/153.1 kB 19.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 954.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823

In [ ]:
import whisper
from transformers import WhisperForConditionalGeneration
import torch
from tqdm import tqdm
import os

# using pickle to serialize the map_dict
import pickle

from huggingface_hub import hf_hub_download
filename = "pytorch_model.bin"
hf_hub_download(repo_id="sanchit-gandhi/whisper-small-hi", filename=filename, local_dir="/content/")


'/content/pytorch_model.bin'

In [ ]:
# to enable verbose printing of exceptions (+ layers matching name)
DEBUG = False

# set to True if your custom model has been trained using DDP (multi-gpu)
# as in my case, in the custom HF model, keys have a prefix (model.)
# it should come from the fact that I have trained on a milti-gpu machine, using DDP
DDP_TRAINED = True

# if DDP we have to add a prefix to match with the HF state_dict
if DDP_TRAINED:
    PREFIX = "model."
else:
    PREFIX = ""

# for now, tested only with medium
MODEL_SIZE = "small"

# the device where you're running this code
DEVICE = "cpu"

# the name of the file with your fine-tuned model
FINETUNED_MODEL = "pytorch_model.bin"


# the name of the file for the serialized map_dict
# a different name, to avoid overwrite it
FILE_DICT = "map_dict.pkl"


In [ ]:

def import_hf_model(finetuned_model, debug=False, model_size="small", device="cpu", file_dict="map_dict.pkl"):

  def has_numbers(inputString):
      return any(char.isdigit() for char in inputString)

  # next functions are used to make sanity checks for the mappings

  # get if it is encoder or decoder
  def extract_function(key_name):
      # encoder or decoder is the first part of the key
      first_part = key_name.split(".")[0]

      key_func = None
      if first_part in ["enconder", "decoder"]:
          key_func = first_part

      return key_func

  def extract_layer_num(key_name):
      # layer num is the third piece
      layer_num = None

      if has_numbers(key_name):
          layer_num = key_name.split(".")[2]

      return layer_num

  # check that the two keys are for layers
  # with the same function
  # (both encoder or both decoder)
  # and have the same layer number
  # this way we are super-safe (I think)
  def sanity_check(key1, key2):
      is_ok = True

      # check same func (encoder or decoder)
      func1 = extract_function(key1)
      func2 = extract_function(key2)

      if func1 != func2:
          print(f"Warning: layers seem to have different functions: {key1},{key2}")
          is_ok = False

      # check same layer_num
      layer1 = extract_layer_num(key1)
      layer2 = extract_layer_num(key2)

      if layer1 != layer2:
          print(f"Warning: layers seem to have different numbers: {key1},{key2}")
          is_ok = False

      return is_ok

  if not os.path.isfile(file_dict):
    # Vanilla means: not custom trained
    print()
    print("Loading vanilla Whisper model")
    model = whisper.load_model(model_size, device=device)

    print("Loading vanilla HF Model")
    hugging_face_model = WhisperForConditionalGeneration.from_pretrained(
        "openai/whisper-" + model_size
    )

    # extract state-dict from both
    state_d_openai = model.state_dict()
    state_d_huggingface = hugging_face_model.model.state_dict()

    # build the mapping between keys...
    map_dict = {}
    print("Matching layers...")

    # for every layer in OpenAI model
    n_sanity_ok = 0

    #
    # here we're considering the cartesian product of the two state dict and try to match
    # rules applied:
    # 1. the two layers have the same shape
    # 2. the two layer have the same parameters' values
    # 3. we apply sanity check (see function above)
    #
    for k in tqdm(state_d_openai):
        # find a layer in the HF model, check with j
        for j in state_d_huggingface:
            # where parameters have same shape and same values
            if state_d_huggingface[j].shape == state_d_openai[k].shape:
                if torch.all(torch.eq(state_d_huggingface[j], state_d_openai[k])).item():
                    # found, register the mapping
                    map_dict[k] = j
                    # make some check and eventually print a warning
                    if sanity_check(k, j) == True:
                        n_sanity_ok += 1

                        # if you enable thsi print you can see the name of the layer
                        # chosen in the match and you will se that they have the same functions
                        if debug:
                            print(k, j)

                    break


    # check if we have matched every entry
    print("Check if we have matched every entry in state_dict...")
    print()
    print(f"Number of keys: {len(map_dict.keys())}")
    assert len(map_dict.keys()) == len(state_d_openai.keys()), "The match is not complete !"

    print(f"Number of sanity_check ok: {n_sanity_ok}")
    print()

    print("Match is complete !!!")
    print()


    # serialize the map_dict to file
    print("Serializing map_dict...")

    with open(file_dict, "wb") as f:
        pickle.dump(map_dict, f)
        f.close()

    print(f"map_dict saved as: {file_dict}...")
    print()


    # # loading with match keys
    # # restart from pickle file
    # print("Reloading map_dict...")
    # print()
    # with open(file_dict, "rb") as f:
    #     map_dict = pickle.load(f)

  # loading fine-tuned dict
  print("Loading fine tuned dict...")

  # added map_location to handle the fact that the custom model has been trained on GPU
  state_dict_finetuned = torch.load(finetuned_model, map_location=torch.device(device))

  # build the state_dict to be used
  # take the key name from standard (OpenAI) and the value from finetuned (HF)
  print("Rebuild the state dict...")
  new_state_dict = {}
  n_except = 0
  for k in tqdm(map_dict.keys()):
      try:
          # You must add "model." if you have used DDP in custom training
          # see DDP_TRAINED above
          # PREFIX is added to a HF fine-tuned 8with DDP). It is not in vanulla HF models
          new_state_dict[k] = state_dict_finetuned[PREFIX + map_dict[k]]
      except:
          n_except += 1

          if debug:
              print(PREFIX + map_dict[k])




  msg_err = f"Rebuild state dict failed, {n_except} pick failed"
  assert n_except == 0, msg_err



  print()
  print("Loading the final model...")
  model.load_state_dict(new_state_dict)
  return model

In [ ]:
model = import_hf_model(finetuned_model=FINETUNED_MODEL)


Loading vanilla Whisper model


100%|███████████████████████████████████████| 461M/461M [00:06<00:00, 77.1MiB/s]


Loading vanilla HF Model


Matching layers...


100%|██████████| 479/479 [00:14<00:00, 32.69it/s]


Check if we have matched every entry in state_dict...

Number of keys: 479
Number of sanity_check ok: 479

Match is complete !!!

Serializing map_dict...
map_dict saved as: map_dict.pkl...

Loading fine tuned dict...
Rebuild the state dict...


100%|██████████| 479/479 [00:00<00:00, 244919.13it/s]


Loading the final model...


In [10]:
model

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-11): 12 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=False)
          (value): Linear(in_features=768, out_features=768, bias=True)
          (out): Linear(in_features=768, out_features=768, bias=True)
        )
        (attn_ln): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
        (mlp_ln): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((768,), eps=1e-0

In [11]:
# Download a sample audio file from Gdrive
import gdown
url = "https://drive.google.com/uc?id=1kKeSvrZo8z5Rsp1q-h3GXpG7vHctKMcG"
output = "sample.wav"
gdown.download(url, output, quiet=False)

# transcription = "ब्रूड बॉक्स लैंगस्ट्रॉथ छत्ते का एक अनिवार्य हिस्सा है।"

Downloading...
From: https://drive.google.com/uc?id=1kKeSvrZo8z5Rsp1q-h3GXpG7vHctKMcG
To: /content/sample.wav
100%|██████████| 197k/197k [00:00<00:00, 3.22MB/s]


'sample.wav'

In [12]:
audio = whisper.load_audio("/content/sample.wav")
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(model.device)


In [13]:
options = whisper.DecodingOptions(fp16 = False, beam_size=5, without_timestamps=True)
result = whisper.decode(model, mel, options)
finetuned_baseline = result.text
finetuned_baseline

'ब्रूड बॉक्स लैंगस्टोट छत्ते का एक अनिवारिय हिस्सा है।'